In [1]:
import pandas as pd
passengers = pd.read_csv("passengers.csv")
print(passengers.shape)
print(passengers.head())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [2]:
# 성별(SEX)는 숫자 데이터 1과 0으로 변환, 여성이 살 확률 높음으로 남성0, 여성1
passengers['Sex'] = passengers['Sex'].map({'female':1,'male':0})
print(passengers.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    0  35.0      0      0   

             Ticket     Fare Cabin Embarked  
0         A/5 21171   7.2500   NaN        S  
1          PC 17599  71.2833   C85        C  
2  STON/O2. 3101282   7.9250   NaN        S  
3            113803  53.1000  C123        S  
4            373450   8.0500   NaN        S  


In [3]:
# Age의 Missing Data는 평균 값으로 대체
passengers['Age'].fillna(value=passengers['Age'].mean(), inplace=True)

In [4]:
# Pclass 1등석, 2등석에 탔는지 각각의 feature로 만들어주기 위해 컬럼을 새로 생성
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 2 else 0)
print(passengers.head(10))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name  Sex        Age  SibSp  \
0                            Braund, Mr. Owen Harris    0  22.000000      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.000000      1   
2                             Heikkinen, Miss. Laina    1  26.000000      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.000000      1   
4                           Allen, Mr. William Henry    0  35.000000      0   
5                                   Moran, Mr. James    0  29.699118      0   
6                            McCarthy, Mr. Timothy

In [5]:
# DataSet 준비
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

In [6]:
# sklearn의 train_test_split을 사용해 학습세트와 평가세트 분리
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

In [7]:
# regularation 데이터 스케일링, sklearn이 제공하는 StandardScaler를 활용해서 손쉽게할 수 있다. 
# StandardScaler는 평균 0, 표준편차 1로 변환하는 방법이지만 이외에도 최소값 0, 최대값 1이 되도록
# 변환하는 MinMaxScaler, 중앙값(median) 0, IQR(interquartile range) 1이 되도록 변환하는 RobustScaler 등이 있다.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [8]:
# 모델 생성 ,데이터만 넣어주고 fit 해주면 끝
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_features, train_labels)
print(model.fit(train_features, train_labels))

LogisticRegression()


In [9]:
# 그러면 이제 학습 세트로 정확도 출력
print(model.score(train_features, train_labels))

0.7874251497005988


In [10]:
# 아까 분리해놓은 테스트 세트에서도 확인
print(model.score(test_features, test_labels))

0.8430493273542601


In [11]:
# 이번엔 각 feature들의 계수(coefficients)를 확인해볼 차례다. 어떤 feature가 생존에 큰 영향을 주는지 확인해볼 수 있으니까.
print(model.coef_)

[[ 1.16488349 -0.41666841  0.89451281  0.46702064]]


In [12]:
# 새로운 데이터 넣어 확인
import numpy as np

Jack = np.array([0.0, 20.0, 0.0, 0.0])
Rose = np.array([1.0, 17.0, 1.0, 0.0])
ME = np.array([0.0, 32.0, 1.0, 0.0])

sample_passengers = np.array([Jack, Rose, ME])

In [13]:
sample_passengers = scaler.transform(sample_passengers)

In [14]:
print(model.predict(sample_passengers))

[0 1 0]


In [15]:
print(model.predict_proba(sample_passengers))

[[0.86475317 0.13524683]
 [0.05845249 0.94154751]
 [0.52988666 0.47011334]]
